## fetch domains for hypothetical proteins from PFAM

In [2]:
import sys,os,shutil,subprocess
import random
import glob, time
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from snpgenie import tools, app, trees
from prody import *

In [3]:
g = tools.genbank_to_dataframe('LT708304_latest.gb')
g = g[g.feat_type!='gene']
g['gene']=g.gene.fillna('')
g=g[1:]
print (len(g))
x=g[(g.gene.str.contains('PE_PGRS') | g.gene.str.contains('PPE')) & (g.translation.str.len()>10)]
#for i,r in x.iterrows():
    #print ('LT708304.1','\t', r.start, '\t', r.end, r.gene, r.locus_tag)   

4199


In [17]:
g.iloc[0]

locus_tag                                            BQ2027_MB0001
codon_start                                                      1
product             CHROMOSOMAL REPLICATION INITIATOR PROTEIN DNAA
transl_table                                                    11
note             Mb0001, dnaA, len: 507 aa. Equivalent to Rv000...
db_xref                                                 GOA:P49991
translation      MTDDPGSGFTTVWNAVVSELNGDPKVDDGPSSDANLSAPLTPQQRA...
gene                                                          dnaA
protein_id                                              SIT98333.1
old_locus_tag                                               Mb0001
id                                                      LT708304.1
start                                                            0
end                                                           1524
strand                                                           1
feat_type                                                     

In [4]:
def find_pfam_domains(g, res=None):
    """Find pfam domains"""
    
    new=[]
    for i,r in g.iterrows():
        if res!=None:
            if r.locus_tag in list(res.locus_tag):
                continue
        print (r.locus_tag)
        try:
            d = searchPfam(r.translation)
        except Exception as e:
            print (e)
            continue
        found = []
        for k in d:
            found = d[k]       
            found['locus_tag'] = r.locus_tag
            found['length'] = len(r.translation)
            found['product'] = r['product']
            new.append(found)    
    df=pd.DataFrame(new) 
    if len(df)>0 and g is not None:
        locs = df['locations'].apply(pd.Series)
        df=pd.concat([df, locs], axis=1)
        df=df.drop(columns=['locations'])
    df = pd.concat([res,df])
    return df

In [63]:
labels=['hypothetical protein','conserved protein','conserved hypothetical',
        'unknown protein','uncharacterized','hypothetical alanine']
def find_unknown(x):
    for l in labels:
        if l in str(x).lower():           
            return True
    return False

g['hypothetical'] = g['product'].apply(find_unknown,1)
h = g[g.hypothetical==True]
print (len(h))

467


In [ ]:
#known = pd.read_csv('hypothetical_pfam.csv')
res = find_pfam_domains(h)
res.to_csv('hypothetical_pfam.csv',index=False)

In [9]:
res = pd.read_csv('hypothetical_pfam.csv')

In [44]:
known = res[~res.id.str.contains('DUF')]
cols=['locus_tag','translation','protein_id','gene']
known = known.merge(g[cols],on='locus_tag')

188


In [49]:
len(h),len(res.locus_tag.unique()),len(known.locus_tag.unique())

(467, 188, 104)

In [61]:
known[:6][['locus_tag','product','id','accession','start','end','length']]

,locus_tag,product,id,accession,start,end,length
0,BQ2027_MB0028,conserved hypothetical protein,T7SS_ESX_EspC,PF10824.8,1,99,105
1,BQ2027_MB0037c,conserved protein,MDMPI_N,PF11716.8,11,147,257
2,BQ2027_MB0037c,conserved protein,Wyosine_form,PF08608.12,183,234,257
3,BQ2027_MB0037c,conserved protein,DinB_2,PF12867.7,10,165,257
4,BQ2027_MB0060,HYPOTHETICAL PROTEIN,DarT,PF14487.6,29,230,230
5,BQ2027_MB0081c,HYPOTHETICAL PROTEIN,AbiEii,PF08843.11,9,196,197


In [39]:
for i,df in known.groupby('locus_tag'):
    print (i)
    print (df[['id','accession','start','end']])
    print (df.iloc[0].translation)

BQ2027_MB0028
              id  accession  start  end
0  T7SS_ESX_EspC  PF10824.8      1   99
MTDRIHVQPAHLRQAAAHHQQTADYLRTVPSSHDAIRESLDSLGPIFSELRDTGRELLELRKQCYQQQADNHADIAQNLRTSAAMWEQHERAASRSLGNIIDGSR
BQ2027_MB0037c
             id   accession  start  end
1       MDMPI_N   PF11716.8     11  147
2  Wyosine_form  PF08608.12    183  234
3        DinB_2   PF12867.7     10  165
MADPGPFVADLRAESDDLDALVAHLPADRWADPTPAPGWTIAHQIGHLLWTDRVALTAVTDEAGFAELMTAAAANPAGFVDDAATELAAVSPAELLTDWRVTRGRLHEELLAVPDGRKLAWFGPPMSAASMATARLMETWAHGLDVADALGVIRPATQRLRSIAHLGVRTRDYAFIVNNLTPPAEPFLVELRGPSGDTWSWGPSDAAQRVTGSAEDFCFLVTQRRALSTLDVNAVGEDAQRWLTIAQAFAGPPGRGR
BQ2027_MB0060
     id  accession  start  end
4  DarT  PF14487.6     29  230
MITRYKPESGFVARSGGPDRKRPHDWIVWHFTHADNLPGIITAGRLLADSAVTPTTEVAYNPVKELRRHKVVAPDSRYPASMASDHVPFYIAARSPMLYVVCKGHSGYSGGAGPLVHLGVALGDIIDADLTWCASDGNAAASYTKFSRQVDTLGTFVDFDLLCQRQWHNTDDDPNRQSRRAAEILVYGHVPFELVSYVCCYNTETMTRVRTLLDPVGGVRKYVIKPGMYY
BQ2027_MB0081c
       id   accession  start  end
5  AbiEii  